# Vector Search

In [22]:
from pathlib import Path
import sys

ROOT_DIR = Path(".").absolute().parent.parent
sys.path.append(str(ROOT_DIR))

In [2]:
import torch

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device `{device}`")

Using device `cpu`


### Q1. Getting the embedding model

In [6]:
from sentence_transformers import SentenceTransformer

In [5]:
model_name = "multi-qa-distilbert-cos-v1"
embedding_model = SentenceTransformer(model_name, device=device)

In [7]:
user_question = "I just discovered the course. Can I still join it?"
generated_embedding = embedding_model.encode(user_question)

# what's the first value of a resulting vector?
generated_embedding[0]

0.078222655

### Prepare the documents

In [23]:
import json

import numpy as np
from tqdm import tqdm

In [8]:
DATA_DIR = Path(ROOT_DIR, "data")

In [12]:
# read docs with IDs
with open(Path(DATA_DIR, "documents-with-ids.json")) as f_in:
    documents = json.load(f_in)

In [18]:
documents_subset = [i for i in documents if i.get("course", "") == "machine-learning-zoomcamp"]
print(f"# of subset documents: {len(documents_subset):,d}")

# of subset documents: 375


### Q2. Creating the embeddings

In [24]:
# generate embeddings for each doc from subset:
embeddings = []
for doc in tqdm(documents_subset):
    question_i, text_i = doc.get("question"), doc.get("text")
    qa_text_i = f"{question_i} {text_i}"
    embed_i = embedding_model.encode(qa_text_i)
    embeddings.append(embed_i)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 375/375 [00:11<00:00, 31.58it/s]


In [25]:
# to matrix
X = np.array(embeddings)
X.shape

(375, 768)

### Q3. Search

In [32]:
scores = X.dot(generated_embedding)
scores.max()

0.6506574

### Vector Search

In [34]:
class VectorSearchEngine():
    def __init__(self, documents: list[dict], embeddings: np.ndarray):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query: np.ndarray, num_results: int = 10):
        scores = self.embeddings.dot(v_query)
        top_idx = np.argsort(-scores)[:num_results]
        top_docs = [self.documents[i] for i in top_idx]
        return top_docs

In [36]:
user_question

'I just discovered the course. Can I still join it?'

In [35]:
# check vector search engine:
search_engine = VectorSearchEngine(documents=documents_subset, embeddings=X)
search_engine.search(v_query=generated_embedding, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

### Q4. Hit rate for our search engine

In [37]:
import pandas as pd

In [45]:
# load ground truth dataset:
df_ground_truth = pd.read_csv(Path(DATA_DIR, "ground-truth-data.csv"))

# keep only #machine-learning-zoomcamp
df_ground_truth = df_ground_truth[
    df_ground_truth.course == "machine-learning-zoomcamp"
].reset_index(drop=True).copy()
ground_truth = df_ground_truth.to_dict(orient='records')

In [58]:
print(f"# of Ground Truth documents: {len(ground_truth):,d}")
ground_truth[0]

# of Ground Truth documents: 1,830


{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [52]:
def hit_rate(relevance_total):
    cnt = 0
    
    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [53]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [56]:
def evaluate(ground_truth, search_engine, num_search_results: int = 5) -> dict:
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id_i, question_i = q["document"], q["question"]
        embed_i = embedding_model.encode(question_i)
        results_i = search_engine.search(embed_i, num_results=num_search_results)
        relevance_i = [d['id'] == doc_id_i for d in results_i]
        relevance_total.append(relevance_i)

    return {
        "hit_rate": hit_rate(relevance_total),
        "merr": mrr(relevance_total),
    }

In [57]:
evaluate(ground_truth, search_engine)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1830/1830 [04:05<00:00,  7.45it/s]


{'hit_rate': 0.9398907103825137, 'merr': 0.8516484517304189}

### Q5. Indexing with ElasticSearch

In [77]:
from elasticsearch import Elasticsearch


ES_CLIENT_HOST = "http://localhost:9200"

In [78]:
!curl {ES_CLIENT_HOST}

{
  "name" : "7087af81676a",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "lUhtIKk4QI-D3H3WM0we7w",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [81]:
es_client = Elasticsearch(ES_CLIENT_HOST)

In [119]:
# config ES:
EMBEDDING_DIM = X.shape[1]

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": EMBEDDING_DIM,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

# Create or Replace index:
if es_client.indices.exists(index=index_name):
    es_client.indices.delete(index=index_name)

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [121]:
# Index docs with embeddings in ElasticSearch:
assert X.shape[0] == len(documents_subset), "Wrong number of Documents / Embeddings"

for doc_i, embed_i in tqdm(zip(documents_subset, X)):
    doc_i["question_text_vector"] = embed_i
    es_client.index(index=index_name, document=doc_i)

375it [00:01, 222.44it/s]


In [122]:
def elastic_search_knn(
    vector: np.ndarray,
    field: str = "question_text_vector",
    course: str = "machine-learning-zoomcamp",
    num_search_results: int = 5
):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": num_search_results,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [123]:
user_question

'I just discovered the course. Can I still join it?'

In [124]:
es_search_result = elastic_search_knn(vector=generated_embedding)
# most popular doc:
es_search_result[0]

{'question': 'The course has already started. Can I still join it?',
 'course': 'machine-learning-zoomcamp',
 'section': 'General course-related questions',
 'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
 'id': 'ee58a693'}

### Q6. Hit-Rate with ElasticSearch

In [127]:
def evaluate(ground_truth, search_engine, num_search_results: int = 5) -> dict:
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id_i, question_i = q["document"], q["question"]
        embed_i = embedding_model.encode(question_i)
        
        results_i = search_engine(embed_i, num_search_results=num_search_results)
        relevance_i = [d["id"] == doc_id_i for d in results_i]
        relevance_total.append(relevance_i)

    return {
        "hit_rate": hit_rate(relevance_total),
        "merr": mrr(relevance_total),
    }

In [128]:
# Hit Rate with Elastic Search (field: `question_text_vector`)
evaluate(ground_truth, elastic_search_knn)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1830/1830 [00:58<00:00, 31.38it/s]


{'hit_rate': 0.9398907103825137, 'merr': 0.8516484517304189}

<hr>